In [22]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.document_loaders import WebBaseLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.documents import Document
from langchain.chains import create_retrieval_chain
from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.messages import HumanMessage, AIMessage

output_parser = StrOutputParser()
llm = ChatOpenAI()
embeddings = OpenAIEmbeddings()
text_splitter = RecursiveCharacterTextSplitter()

In [15]:
loader = WebBaseLoader("https://www.enterprisesg.gov.sg/financial-support/enterprise-development-grant")

docs = loader.load()

In [16]:
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [17]:
prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [18]:
retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [20]:
#retrieval_chain.invoke({"input": "how can langsmith help with testing?"})
response = retrieval_chain.invoke({"input": "How do I check if I'm eligible for the Enterprice Development grant?"})
print(response["answer"])

To check if you are eligible for the Enterprise Development Grant, you need to ensure that your business entity is registered and operating in Singapore, has at least 30% local equity held by Singaporean(s) and/or Singapore PR(s), and is financially ready to start and complete the project. Additionally, you can refer to the eligibility criteria mentioned in the provided context for more details.


In [23]:
prompt = ChatPromptTemplate.from_messages([
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
    ("user", "Given the above conversation, generate a search query to look up to get information relevant to the conversation")
])
retriever_chain = create_history_aware_retriever(llm, retriever, prompt)

In [24]:
chat_history = [HumanMessage(content="Can Singaporean companies apply for the EDG?"), AIMessage(content="Yes!")]
retriever_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

[Document(page_content='Enterprise Development Grant (EDG)\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nx \n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\nGrow Your Business\n\nBoost Capabilities\n\nOverview\n\n\nGrowth & Transformation\n\n\nProductivity & Digitalisation\n\n\nTalent Attraction & Development\n\n\nQuality & Standards\n\n\nSustainability\n\n\n\n\nInnovate with Us\n\nOverview\n\n\nInnovation Capabilities\n\n\nInnovation Talent\n\n\nMarket Access & Networks\n\n\n\n\nGo Global\n\nOverview\n\n\nMarket Guides\n\n\nMarket Entry Support\n\n\nInternational Agreements\n\n\n\n\nPartner with Singapore\n\nWhy Singapore\n\n\nInnovation & Startups\n\n\nTrade\n\n\nInfrastructure\n\n\n\n\nFake Navigation Item\n\n\n\n\nIndustries\n\n\nFinancial Support\n\n\nResources\n\n\nAbout Us\n\n\n\n\n\n \n\n\n\n\n\n \n\n\n\n \n\n\n\n\nGrow Your Business\n\nBoost Capabilities\n\nOverview\n\n\nGrowth & Transformation\n\n\nProductivity & Digitalisati

In [25]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "Answer the user's questions based on the below context:\n\n{context}"),
    MessagesPlaceholder(variable_name="chat_history"),
    ("user", "{input}"),
])
document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever_chain, document_chain)

In [28]:
chat_history = [HumanMessage(content="Can Singaporean companies apply for the EDG?"), AIMessage(content="Yes!")]
retrieval_chain.invoke({
    "chat_history": chat_history,
    "input": "Tell me how"
})

{'chat_history': [HumanMessage(content='Can Singaporean companies apply for the EDG?'),
  AIMessage(content='Yes!')],
 'input': 'Tell me how',
 'context': [Document(page_content='Enterprise Development Grant (EDG)\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nx \n\n\n\n\n\n\n\n \n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n \n\n\n\n\nGrow Your Business\n\nBoost Capabilities\n\nOverview\n\n\nGrowth & Transformation\n\n\nProductivity & Digitalisation\n\n\nTalent Attraction & Development\n\n\nQuality & Standards\n\n\nSustainability\n\n\n\n\nInnovate with Us\n\nOverview\n\n\nInnovation Capabilities\n\n\nInnovation Talent\n\n\nMarket Access & Networks\n\n\n\n\nGo Global\n\nOverview\n\n\nMarket Guides\n\n\nMarket Entry Support\n\n\nInternational Agreements\n\n\n\n\nPartner with Singapore\n\nWhy Singapore\n\n\nInnovation & Startups\n\n\nTrade\n\n\nInfrastructure\n\n\n\n\nFake Navigation Item\n\n\n\n\nIndustries\n\n\nFinancial Support\n\n\nResources\n\n\nAbout Us\n\n